In [2]:
#
# USAR OLLAMA API MEDIANTE SERVEO Y REVERSETUNNEL (USAR OLLAMA LOCAL Y USAR MODELOS DEL CUADERNO)
#
import fileinput
import shutil
import subprocess
import os
import time
import subprocess
import random
import re
# Configuración inicial
OLLAMA_MODELS = "deepseek-coder-v2"
num_ctx_models = 20000
root_password = "root"
def cambiar_contraseña_root():
    # Comando para cambiar la contraseña del usuario root a 'root'
    comando = ['chpasswd']

    # Entrada para el comando chpasswd en el formato 'usuario:contraseña'
    entrada = f'root:{root_password}'


    try:
        # Ejecutar el comando con la entrada proporcionada
        resultado = subprocess.run(comando, input=entrada, text=True, capture_output=True, check=True)
        print("Contraseña cambiada exitosamente.")
        print("Salida del comando:", resultado.stdout)
    except subprocess.CalledProcessError as e:
        print("Error al cambiar la contraseña:", e)
        print("Salida de error:", e.stderr)

# Llamar a la función para cambiar la contraseña
cambiar_contraseña_root()

# Ruta del archivo de configuración de OpenSSH
config_path = "/etc/ssh/sshd_config"
backup_path = "/etc/ssh/sshd_config.bak"


def install_openssh_server():
    """Instala el servidor OpenSSH si no está instalado."""
    print("Instalando OpenSSH server...")
    subprocess.run(["apt", "install", "openssh-server", "-y"], check=True)


def edit_sshd_config():
    """Edita el archivo sshd_config para habilitar acceso root y autenticación por contraseña."""
    print("Creando copia de seguridad de sshd_config...")
    shutil.copy(config_path, backup_path)

    print("Modificando sshd_config...")
    with fileinput.FileInput(config_path, inplace=True) as file:
        for line in file:
            if line.strip().startswith("#PermitRootLogin") or line.strip().startswith("PermitRootLogin"):
                print("PermitRootLogin yes")  # Permitir acceso como root
            elif line.strip().startswith("#PasswordAuthentication") or line.strip().startswith("PasswordAuthentication"):
                print("PasswordAuthentication yes")  # Permitir autenticación por contraseña
            elif line.strip().startswith("#PubkeyAuthentication") or line.strip().startswith("PubkeyAuthentication"):
                print("PubkeyAuthentication no")  # Deshabilitar autenticación con claves públicas
            else:
                print(line, end="")  # Mantener las líneas no relevantes sin cambios


def restart_ssh_service():
    """Reinicia el servicio SSH para aplicar los cambios."""
    print("Reiniciando servicio SSH...")
    try:
        subprocess.run(["service", "ssh", "restart"], check=True)
        print("Servicio SSH reiniciado exitosamente.")
    except subprocess.CalledProcessError as e:
        print(f"Error al reiniciar el servicio SSH: {e}")



def run(commands):
    for command in commands:
        subprocess.run(command, shell=True, check=True)


def install_ollama():
    """Instala Ollama si no está instalado."""
    ollama_installed = subprocess.run(["which", "ollama"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if ollama_installed.returncode != 0:
        print("Ollama no encontrado. Instalando...")
        commands = [
            "curl -fsSL https://ollama.com/install.sh | sh",
        ]
        run(commands)
    else:
        print("Ollama ya está instalado, omitiendo la instalación.")


def start_ollama_server():
    """Inicia el servidor Ollama en segundo plano."""
    print("Iniciando el servidor Ollama...")
    os.system("export OLLAMA_HOST=0.0.0.0")
    subprocess.Popen(["/usr/local/bin/ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)  # Espera para que Ollama esté activo


def pull_ollama_models(models):
    """Ejecuta 'ollama pull' para descargar varios modelos."""
    model_list = [model.strip() for model in models.split(",")]
    for model in model_list:
        print(f"Ejecutando 'ollama pull {model}'...")
        subprocess.run(["/usr/local/bin/ollama", "pull", model], check=True)


def start_serveo_tunnel():
    """Inicia un túnel SSH a través de Serveo y maneja errores de puertos."""
    while True:
        # Generar un puerto aleatorio de 4 dígitos
        puerto_serveo = random.randint(1024, 9999)
        print(f"Iniciando túnel SSH con Serveo en el puerto {puerto_serveo}...")

        # Comando para iniciar el túnel
        command = [
            "ssh", 
            "-o", "StrictHostKeyChecking=no", 
            "-R", f"{puerto_serveo}:localhost:22", 
            "serveo.net"
        ]

        try:
            # Ejecutar el comando y leer la salida en tiempo real
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            for line in iter(process.stdout.readline, ""):
                print(line.strip())  # Mostrar la salida en tiempo real

                # Verificar si el túnel se creó correctamente
                if re.search(rf"Forwarding TCP connections from serveo.net:{puerto_serveo}", line):
                    print(f"Túnel creado exitosamente en serveo.net:{puerto_serveo}")
                    process.wait()  # Esperar a que el proceso termine
                    return

                # Si el puerto es menor a 1024
                if "Public forwarding of ports lower than 1024 is disallowed" in line:
                    print("Error: El puerto debe ser mayor a 1024. Intentando con otro puerto...")
                    process.terminate()
                    break

        except Exception as e:
            print(f"Ocurrió un error: {e}. Intentando con otro puerto...")
            if process.poll() is None:
                process.terminate()


def create_and_execute_model_files(models):
    """Crea y ejecuta archivos Modelfile para cada modelo en la lista."""
    model_list = [model.strip() for model in models.split(",")]
    for model in model_list:
        modelfile_content = f"""FROM {model}\nPARAMETER num_ctx {num_ctx_models}"""
        modelfile_name = "Modelfile"

        # Crear archivo Modelfile
        with open(modelfile_name, "w") as modelfile:
            modelfile.write(modelfile_content)

        print(f"Archivo {modelfile_name} creado para el modelo {model}.")

        # Ejecutar el comando ollama create
        command = f"/usr/local/bin/ollama create -f {modelfile_name} {model}-extra-ctx"
        print(f"Ejecutando: {command}")
        subprocess.run(command, shell=True, check=True)

        # Eliminar archivo Modelfile
        os.remove(modelfile_name)
        print(f"Archivo {modelfile_name} eliminado para el modelo {model}.")


def main():
    # Instalación y configuración inicial
    install_openssh_server()
    edit_sshd_config()
    restart_ssh_service()

    install_ollama()

    # Iniciar servicios
    start_ollama_server()
    pull_ollama_models(OLLAMA_MODELS)
    os.system("clear")  # Limpia la pantalla después de descargar los modelos

    # Crear y ejecutar archivos Modelfile para cada modelo
    create_and_execute_model_files(OLLAMA_MODELS)

    # Iniciar túnel Serveo
    start_serveo_tunnel()


if __name__ == "__main__":
    main()


Contraseña cambiada exitosamente.
Salida del comando: 
Instalando OpenSSH server...
Creando copia de seguridad de sshd_config...
Modificando sshd_config...
Reiniciando servicio SSH...
Servicio SSH reiniciado exitosamente.
Ollama no encontrado. Instalando...
Iniciando el servidor Ollama...
Ejecutando 'ollama pull deepseek-coder-v2'...
Archivo Modelfile creado para el modelo deepseek-coder-v2.
Ejecutando: /usr/local/bin/ollama create -f Modelfile deepseek-coder-v2-extra-ctx
Archivo Modelfile eliminado para el modelo deepseek-coder-v2.
Iniciando túnel SSH con Serveo en el puerto 4924...
Forwarding TCP connections from serveo.net:4924
Túnel creado exitosamente en serveo.net:4924


KeyboardInterrupt: 

In [2]:
#
# USAR NGROK PARA USAR URL DE OLLAMA SERVE
#
import os
import subprocess
import time

# Configuración inicial
NGROK_TOKEN = "NGROK_TOKEN"
OLLAMA_MODELS = "llama3.2-vision"
NGROK_PORT = 11434
num_ctx_models = 20000

def run(commands):
    for command in commands:
        subprocess.run(command, shell=True, check=True)


def install_ollama():
    """Instala Ollama si no está instalado."""
    ollama_installed = subprocess.run(["which", "ollama"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if ollama_installed.returncode != 0:
        print("Ollama no encontrado. Instalando...")
        commands = [
            "curl -fsSL https://ollama.com/install.sh | sh",
        ]
        run(commands)
    else:
        print("Ollama ya está instalado, omitiendo la instalación.")


def install_ngrok():
    """Descarga e instala ngrok."""
    print("Descargando e instalando ngrok...")
    os.system("wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz")
    os.system("apt install -y tar")
    os.system("tar -xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin")


def start_ollama_server():
    """Inicia el servidor Ollama en segundo plano."""
    print("Iniciando el servidor Ollama...")
    os.system("export OLLAMA_HOST=0.0.0.0")
    subprocess.Popen(["/usr/local/bin/ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)  # Espera para que Ollama esté activo


def pull_ollama_models(models):
    """Ejecuta 'ollama pull' para descargar varios modelos."""
    model_list = [model.strip() for model in models.split(",")]
    for model in model_list:
        print(f"Ejecutando 'ollama pull {model}'...")
        subprocess.run(["/usr/local/bin/ollama", "pull", model], check=True)


def configure_ngrok(token):
    """Configura ngrok con el token de autenticación."""
    print("Configurando ngrok con el token de autenticación...")
    os.system(f"ngrok config add-authtoken {token}")


def start_ngrok(port):
    """Inicia ngrok en el puerto especificado."""
    print(f"Iniciando ngrok para el puerto {port}...")
    subprocess.Popen(["ngrok", "http", str(port), f"--host-header=localhost:{port}"],
                     stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)  # Espera para que ngrok esté activo


def create_and_execute_model_files(models):
    """Crea y ejecuta archivos Modelfile para cada modelo en la lista."""
    model_list = [model.strip() for model in models.split(",")]
    for model in model_list:
        modelfile_content = f"""FROM {model}\nPARAMETER num_ctx {num_ctx_models}"""
        modelfile_name = "Modelfile"

        # Crear archivo Modelfile
        with open(modelfile_name, "w") as modelfile:
            modelfile.write(modelfile_content)

        print(f"Archivo {modelfile_name} creado para el modelo {model}.")

        # Ejecutar el comando ollama create
        command = f"/usr/local/bin/ollama create -f {modelfile_name} {model}--extra-ctx"
        print(f"Ejecutando: {command}")
        subprocess.run(command, shell=True, check=True)

        # Eliminar archivo Modelfile
        os.remove(modelfile_name)
        print(f"Archivo {modelfile_name} eliminado para el modelo {model}.")


def get_ngrok_status():
    """Obtiene el estado del túnel ngrok."""
    print("Obteniendo el estado del túnel ngrok...")
    response = os.popen("curl -s http://127.0.0.1:4040/api/tunnels").read()
    print("Estado del túnel ngrok:")
    print(response)


def main():
    # Instalación y configuración
    install_ollama()
    install_ngrok()

    # Iniciar servicios
    start_ollama_server()
    pull_ollama_models(OLLAMA_MODELS)
    os.system("clear")  # Limpia la pantalla después de descargar los modelos

    # Crear y ejecutar archivos Modelfile para cada modelo
    create_and_execute_model_files(OLLAMA_MODELS)
    configure_ngrok(NGROK_TOKEN)
    start_ngrok(NGROK_PORT)
    # Obtener estado
    get_ngrok_status()


if __name__ == "__main__":
    main()


Ollama no encontrado. Instalando...
Descargando e instalando ngrok...
Iniciando el servidor Ollama...
Ejecutando 'ollama pull llama3.2-vision'...
Archivo Modelfile creado para el modelo llama3.2-vision.
Ejecutando: /usr/local/bin/ollama create -f Modelfile llama3.2-vision--extra-ctx
Archivo Modelfile eliminado para el modelo llama3.2-vision.
Configurando ngrok con el token de autenticación...
Iniciando ngrok para el puerto 11434...
Obteniendo el estado del túnel ngrok...
Estado del túnel ngrok:
{"tunnels":[{"name":"command_line","ID":"02d003d63349fef2acfcbfacc503898e","uri":"/api/tunnels/command_line","public_url":"https://ead4-34-29-193-138.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:11434","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}



In [ ]:
!curl http://127.0.0.1:4040

# Información del Cuaderno

Este cuaderno fue adaptado por [HirCoir](https://www.youtube.com/@HirCoir).

¡Suscríbete a su canal para más contenido interesante!
